In [66]:
#create google map client with api key
import gmaps
import gmaps.datasets
import googlemaps
api_key = open("../shared/shared_google_api_key.txt").readline()
gmaps.configure(api_key=api_key) # Your Google API key
google_map_api = googlemaps.Client(key=api_key)


In [ ]:
#geocode neighborhoods
import geocoder
import pandas
nyc_neighborhoods = pandas.read_csv("./street_easy_ny_neighborhoods.csv")
for index, row in nyc_neighborhoods.iterrows():
    if index % 5 == 0:
        print(str(index) + " neighborhoods geocoded" )
    current_location = geocoder.google(row["neighborhood"] + ", NY")
    if len(current_location.latlng) > 0:
        nyc_neighborhoods.set_value(index, "Lat", current_location.latlng[0])
        nyc_neighborhoods.set_value(index, "Long", current_location.latlng[1])
print("geocoding complete")

#filter neighborhoods to new york lat/long
nyc_neighborhoods = nyc_neighborhoods[
(nyc_neighborhoods["Lat"] >= 40) &
(nyc_neighborhoods["Lat"] <= 41) &
(nyc_neighborhoods["Long"] <= -73) &
(nyc_neighborhoods["Long"] >= -75)
]
#save to avoid redoing geocoding
nyc_neighborhoods.to_csv('./nyc_neighborhoods_with_location.csv')


In [17]:
#load geocoded neighborhoods
import pandas
nyc_neighborhoods = pandas.read_csv("./nyc_neighborhoods_with_location.csv")

In [63]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import re 
#crawl streat easy for average rent by neighborhood
def get_for_rent_url(location_keyword):
    location_keyword = location_keyword.replace(' ', '-')
    return 'http://streeteasy.com/for-rent/'+location_keyword+"/beds:1?page=1&sort_by=price_asc"

def get_prices(page_lines):
    prices = pd.DataFrame()
    num_prices=0 
    for i in range(0, len(page_lines)):
        #price
        current_list = page_lines[i].find_all('span',{'class':'price'})
        if len(current_list) > 0:
            price = float(current_list[0].string.replace('$','').replace(',',''))
            prices.set_value(num_prices, "price", price)
            num_prices += 1
    return prices

    
for index, row in nyc_neighborhoods.iterrows():
    #format neighborhood for url query string
    neighborhood = row["neighborhood"].replace(" ", "-")
    url=get_for_rent_url(neighborhood)
    if index % 5 == 0:
        print(str(index) + " neighborhoods processed")
    try:
        url_source=urlopen(url).read()
        decoded_source = url_source.decode('utf-8')
        soup=BeautifulSoup(url_source,'html.parser')
        potential_hit = re.search('Browse photos of (\d+)', str(url_source.decode('utf-8')))
        num_listings = potential_hit.group(1)
        page_lines=soup.find_all(lambda tag: tag.has_attr('data-id'))
        prices = get_prices(page_lines)
        average_price = prices["price"].mean()
        nyc_neighborhoods.set_value(index, "average 1BD rent", average_price)
        nyc_neighborhoods.set_value(index, "number of listings", num_listings)

    except Exception as e:
        print("error")
        print(e)
        print(neighborhood + " not found on street easy")
        nyc_neighborhoods.set_value(index, "average 1BD rent", 0)
        nyc_neighborhoods.set_value(index, "number of listings", 0)

nyc_neighborhoods.to_csv('./nyc_neighborhoods_with_locations_and_prices')

0 neighborhoods processed
error
HTTP Error 404: Not Found
Civic-Center not found on street easy
5 neighborhoods processed
error
HTTP Error 404: Not Found
Fulton/Seaport not found on street easy
10 neighborhoods processed
error
HTTP Error 404: Not Found
Lower-East-Side not found on street easy
15 neighborhoods processed
error
HTTP Error 404: Not Found
Stuyvesant-Town/PCV not found on street easy
20 neighborhoods processed
error
HTTP Error 404: Not Found
Turtle-Bay not found on street easy
25 neighborhoods processed
error
HTTP Error 404: Not Found
Hell's-Kitchen not found on street easy
30 neighborhoods processed
35 neighborhoods processed
error
HTTP Error 404: Not Found
Morningside-Heights not found on street easy
40 neighborhoods processed
error
HTTP Error 404: Not Found
Manhattanville not found on street easy
45 neighborhoods processed
error
HTTP Error 404: Not Found
Marble-Hill not found on street easy
50 neighborhoods processed
error
HTTP Error 404: Not Found
Baychester not found on

In [64]:
nyc_neighborhoods.to_csv('./nyc_neighborhoods_with_locations_and_prices')

In [ ]:

import json

json1_file = open('./all_heatmaps.json')
json1_str = json1_file.read()
all_heatmaps = json.loads(json1_str)


In [154]:
import pandas 
import googlemaps

from datetime import datetime
import time
import math
search_radius = 1609.34*.75 #meters
num_locations = 0
def count_places_near_neighborhoods(nyc_neighborhoods, keywords):
    num_locations = 0
    for index, row in nyc_neighborhoods.iterrows():
        if num_locations % 10 == 0:
            print(str(num_locations) + " neigborhoods processed")
        num_locations += 1
        current_location = (row["Lat"], row["Long"])
        places_nearby = google_map_api.places_nearby(location=current_location,keyword= keywords, radius=search_radius)
        nyc_neighborhoods.set_value(index, "number of nearby " + keywords, len(places_nearby))

    return nyc_neighborhoods

def normalize_by_listings(nyc_neighborhoods, keywords):
    for index, row in nyc_neighborhoods.iterrows():
        if row["number of listings"] > 0:
            intensity = row["number of nearby " + keywords]/row["number of listings"]
            nyc_neighborhoods.set_value(
                index,
                keywords + "heatmap intensity", 
                intensity
                )
        else:
            nyc_neighborhoods.set_value(
                index,
                keywords + "heatmap intensity", 
                0
                )
    return nyc_neighborhoods
                                    
def get_heatmap_for_keywords(nyc_neighborhoods, keywords):
    heatmap = []
    for index, row in nyc_neighborhoods.iterrows():
        heatmap.append((
            row["Lat"],
            row["Long"],
            row[keywords + "heatmap intensity"]
        ))
    return heatmap

def get_heatmap_for_rent(nyc_neighborhoods):
    heatmap = []
    for index, row in nyc_neighborhoods.iterrows():
        heatmap.append((
            row["Lat"],
            row["Long"],
            row["average 1BD rent"]
        ))
    return heatmap

business_keywords_list = ["thai restaurant", "yoga"]#, "starbucks", "smoke shop", "checks cashed"]
nyc_neighborhoods = nyc_neighborhoods[:5]
all_heatmaps = {}
for keywords in business_keywords_list:
    print("current keywords: " + keywords)
    nyc_neighborhoods = count_places_near_neighborhoods(nyc_neighborhoods, keywords)
    nyc_neighborhoods = normalize_by_listings(nyc_neighborhoods, keywords)
    all_heatmaps[keywords.replace('heatmap intensity', '')] = get_heatmap_for_keywords(nyc_neighborhoods, keywords)
all_heatmaps["rent"] = get_heatmap_for_rent(nyc_neighborhoods)

current keywords: thai restaurant
0 neigborhoods processed
3
4
4
4
4
[(40.712217200000005, -74.01605759999998, 0.0375), (40.746500399999995, -74.00137370000002, 0.015503875968992248), (40.746500399999995, -74.00137370000002, 0.04597701149425287), (40.715750899999996, -73.9970307, 0.10810810810810811), (40.714051899999994, -74.00283639999998, 0.0)]
current keywords: yoga
0 neigborhoods processed
4
4
4
4
4
[(40.712217200000005, -74.01605759999998, 0.05), (40.746500399999995, -74.00137370000002, 0.015503875968992248), (40.746500399999995, -74.00137370000002, 0.04597701149425287), (40.715750899999996, -73.9970307, 0.10810810810810811), (40.714051899999994, -74.00283639999998, 0.0)]


In [69]:
import json
with open('all_heatmaps.json', 'w') as fp:
    json.dump(all_heatmaps, fp)

In [ ]:
import json

json1_file = open('./all_heatmaps.json')
json1_str = json1_file.read()
all_heatmaps = json.loads(json1_str)


In [155]:

import pandas as pd
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets
from IPython.display import clear_output
import gmaps

def on_refresh_click(google_map):
    google_map = refresh_map(google_map)
    display(google_map)
    print("map refreshed")
    
def add_layer_with_properties(google_map, weighted_heatmap_data, color_gradient, layer_opacity):
    print(weighted_heatmap_data)
    max_intensity = 0
    intensity_scale = 5
    for data_point in weighted_heatmap_data:
        print(data_point)
        if data_point[2] > max_intensity:
            max_intensity = data_point[2]
    #normalize intensities
    for i in range( 0 ,len(weighted_heatmap_data)):
        scaled_intensity = (weighted_heatmap_data[i][2]*1.00/max_intensity)*intensity_scale
        weighted_heatmap_data[i] =  (weighted_heatmap_data[i][0],weighted_heatmap_data[i][1],scaled_intensity)
    
    layer = gmaps.WeightedHeatmap(data= weighted_heatmap_data)
    layer.gradient = color_gradient
    layer.max_intensity = intensity_scale
    layer.point_radius = 35
    layer.opacity = layer_opacity
    google_map.add_layer(layer)  
    
def refresh_map(google_map):
    color_gradients = {
                        "minus": [
                                    (0, 0, 0, 0),
                                    (255, 0, 0, .7),
                                    (255, 0, 0, .8),
                                    (255, 0, 0, .99)
                                ],
                       "plus": [
                                    (0, 0, 0, 0),
                                    (0, 0, 255, .7),
                                    (0, 0, 255, .8),
                                    (0, 0, 255, .99)
                                ]
                    }
    plus_gentrification_keyword = ["thai restaurant", "yoga", "starbucks"]
    minus_gentrification_keyword = ["checks cashed", "99 cent store", "smoke"]
    num_layers = 1
    #add layers based on checkboxes
    for c in cb_container.children:
        
        if c.value:
            num_layers += 1
    #scaling for visibility with multiple layers
    opacity = .7 - num_layers*.065
    for c in cb_container.children:
        if c.value:
            if c.description in plus_gentrification_keyword:
                color_gradient = color_gradients["plus"]
            else:
                color_gradient = color_gradients["minus"]
            add_layer_with_properties(google_map, all_heatmaps[c.description], color_gradient, opacity)
    price_gradient = [(255,255,255,0),(255,255,255,.4),(255,255,255,.7),(255,255,255,.99)]
    add_layer_with_properties(google_map, all_heatmaps["rent"], price_gradient,.5)

    return google_map

checkboxes = []
cb_container = widgets.HBox()
display(cb_container)
for keywords in business_keywords_list:
    checkboxes.append(widgets.Checkbox(description = keywords, value= True, width = 60))
cb_container.children=[i for i in checkboxes]


In [156]:
#refresh map based on checkbox values
google_map = gmaps.Map()
refresh_map(google_map)
display(google_map)


[(40.712217200000005, -74.01605759999998, 0.0375), (40.746500399999995, -74.00137370000002, 0.015503875968992248), (40.746500399999995, -74.00137370000002, 0.04597701149425287), (40.715750899999996, -73.9970307, 0.10810810810810811), (40.714051899999994, -74.00283639999998, 0.0)]
(40.712217200000005, -74.01605759999998, 0.0375)
(40.746500399999995, -74.00137370000002, 0.015503875968992248)
(40.746500399999995, -74.00137370000002, 0.04597701149425287)
(40.715750899999996, -73.9970307, 0.10810810810810811)
(40.714051899999994, -74.00283639999998, 0.0)
[(40.712217200000005, -74.01605759999998, 0.05), (40.746500399999995, -74.00137370000002, 0.015503875968992248), (40.746500399999995, -74.00137370000002, 0.04597701149425287), (40.715750899999996, -73.9970307, 0.10810810810810811), (40.714051899999994, -74.00283639999998, 0.0)]
(40.712217200000005, -74.01605759999998, 0.05)
(40.746500399999995, -74.00137370000002, 0.015503875968992248)
(40.746500399999995, -74.00137370000002, 0.045977011494

In [75]:
google_map = gmaps.Map()
price_gradient = [(255,255,0,0),(255,0,0,.5),(0,0,255,.99)]
add_layer_with_properties(google_map, all_heatmaps["rent"], price_gradient,.7)
display(google_map)

In [ ]:
nyc_neighborhoods.where(nyc_neighborhoods["average 1BD rent"]>0).sort("average 1BD rent", ascending=False)